In [5]:
import pandas as pd
import sqlite3
from tqdm.notebook import tqdm

Tasks: 
* First, build a profile of providers referring patients to the major hospitals in Nashville. Are certain specialties more likely to refer to a particular hospital over the others?
* Determine which professionals Vanderbilt Hospital should reach out to in the Nashville area to expand their own patient volume. 
    - First, research which professionals are sending significant numbers of patients only to competitor hospitals (such as TriStar Centennial Medical Center).
    - Next, consider the specialty of the provider. If Vanderbilt wants to increase volume from Orthopedic Surgeons or from Family Medicine doctors who should they reach out to in those areas?
* Finally, look for "communities" of providers in the Nashville/Davidson County CBSA. Make use of the Louvain community detection algorithm from Neo4j: https://neo4j.com/docs/graph-data-science/current/algorithms/louvain/.

In [31]:
#Reading in DocGraph HOP Teaming data
hops_and_dreams = pd.read_csv('../data/DocGraph_Hop_Teaming_2018.csv', chunksize = 10000)
# List to hold output of loop
hop_chunks = []
# Loop for filtering criteria
for hop in hops_and_dreams:
    # transaction_count <= 50 & average_day_wait < 50
    hop = hop[(hop.transaction_count>= 50) & (hop.average_day_wait< 50)]
    # appending output to hop_chunks list
    hop_chunks.append(hop)


In [34]:
# Concatenating hop_chunks list to hop dataframe
hop = pd.concat(hop_chunks, ignore_index = True)
hop.head()

,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait
0,1508085911,1730166125,58,67,23.925,43.923
1,1508167040,1730166125,51,51,28.196,52.876
2,1508863549,1730166125,340,391,18.302,42.422
3,1508867870,1730166125,50,79,12.658,26.402
4,1508011040,1730166224,132,145,8.579,28.053


In [19]:
hop

,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait
7,1508085911,1730166125,58,67,23.925,43.923
10,1508167040,1730166125,51,51,28.196,52.876
15,1508863549,1730166125,340,391,18.302,42.422
18,1508867870,1730166125,50,79,12.658,26.402
25,1508011040,1730166224,132,145,8.579,28.053
32,1497971881,1730166224,120,135,49.822,61.955
35,1508163577,1730166224,67,127,8.346,26.050
36,1508159674,1730166224,145,212,2.939,10.660
38,1508205808,1730166224,48,64,14.156,35.859
48,1508170853,1730166307,81,115,32.930,56.683
